In [1]:
# what model am i selecting
# why am i selecting the model
# evaluate the results
# use test data
# possibly try another model
# end with conclusion

# Introduction/Project Overview:
In this notebook, I will present my solution and analysis of the Titanic dataset. This is a very famous dataset that can be found on [kaggle](https://www.kaggle.com/c/titanic/data). The dataset contains demographics of the Titanc passengers, incluiding who survived and who did not. The goal is to build a model that can correctly classify new examples (check who will survive or not). Throughout this notebook I will visualize the data, explain some data preprocessing techniques, construct and evaluate models and analyze the results. 

#### Data Exploration & Preprocessing:  
I will conduct a comprehensive exploration of the dataset, analyzing its various features, checking for missing values, and gaining insights into the distribution of variables. Prior to building the models, I will preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features to ensure optimal model performance.

#### Model Building & Evaluation:
In this notebook I will try implement several models to try and correctly classify passengers who survied. This is a supervised learning task as we are given the labels of who survived and who did not. For this project the models I have chosen are logistic regression, decision trees, random forests, support vector machines and neural networks. For each of these modules I will evalute their peformance using f1score, confusion matrices, and overall accuracy. 

#### Conclusion: 
Finally, I will interpret the results of the models, identifying significant factors that contribute to passenger survival prediction and discussing potential areas for model improvement. The Titanic dataset is a good challange to test your knowledege on machine learning. This will serve as a good test for me to keep learning and testing my skills.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Exploration
As mentioned earlier I got the dataset from kaggle. The link to that can be found above. The download came with two csv files. One for the training set and one for the test set. Since I have it locally on my computer I can eassily access the data as shown below. Some of the first steps we will do before creating a model is to see what our data looks like.

In [3]:
# read train and test sets
train = pd.read_csv('./train.csv') 
test = pd.read_csv('./test.csv') 

We loaded in the data into pandas dataframes and now we want to see what our data looks like. What does vairables does it contain and what data types, etc. First lets start by getting its size. 

In [4]:
train.info() # get info on our train 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


From above we see that we have a total of 891 samples. This is a good ammount as it is a big enohgh ammount for the model to learn from but not too large where we would require lots of computing power and time for training. We also see that we have 12 columns.`Age` has 177 missing values which is a big ammount. `Cabin` has a lot of missing values too. 

What we will do now is try and fill in the missing values for age so lets try to explore our data first to see what it contains. 

In [5]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Lets count our missing values

In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

We see that age has a good ammount of missing values. Cabin does too however it might not be very useful when making a prediction on who survives the titanic. Lets breakdown each column, what they mean and if we will keep them. 

`PassengderId`: This column is not useful because its just a id assigned by the dataset. We will drop this column before training. 

`Survived`: This column is our labels and is important since this is a supervised machine learning problem. If we wanted to go with a clustering/unsupervised learning we could drop this. For the purpose of the notebook we will be keeping this. 

`Pclass`: On kaggle it says that this columns serves "A proxy for socio-economic status". Where 1 is upper, 2 is middle and 3 is upper. This will be important for our model

`Name`: This is simply the name of the passenger. This could be important as some family last names could mean that they are from a wealthier family and therfore might have a higher chance of surviving. We can also use this to approximate age. For this notebook we will most likely drop it. 

`Sex/Age`: The sex of the passenger. Since women and children were  prioritized in the case of an emergency this would helpful to determined who would survive. 

`SibSp`: In the kaggle description of the dataset it says that sibsp is "# of siblings / spouses aboard the Titanic". This could be helpful. 

`Parch`: In the kaggle description of the dataset it says that parch is "# of parents / children aboard the Titanic". This will also be helpful. 

`Ticket`: Simply the ticket ID so we can remove this.

`Fare`: How much they paid for their ticket. This can be useful as maybe workers did not pay for ticket and upper class people payed for their tickets. We will keep this column. 

`Cabin`: This is the cabin they were staying at. This could be useful but there are several missing values in this column so for now we will ignore it. 

`Embarked`: The Location of where they embarked. This could be important to determine who survived. For examples people who embarked at a certain location might be workers and others might be upper class familiies. This could be useful. 

The columns we will be dropping are `PassengerID`, `Ticket`, and `Cabin`. We will keep `Name` for now because it can help us with filling in missing age values. 

In [7]:
# drop the columns PassengerId, Name, Ticket, Cabin
train.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [8]:
train.info() # get info on our train data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Fare      891 non-null    float64
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [9]:
train = train.dropna(subset=['Embarked']) # drop rows with missing embarked values

In [10]:
train.isnull().sum() # count missing values again

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

Now that we got rid of the columns that we did not need, we are going to fill in the missing values for `Age`. Most names contain a `Mr` or  `Mrs` in them. These people would be older because they are married so what we can do is get the average age of people with `Mr` or `Mrs` in their name and use that for their missing `Age` value.  

If they don't have that in their name and their `Parch` is zero then their age will be 0. This is because they are not a parent or child and in the kaggle datset info we are told that "Some children travelled only with a nanny, therefore parch=0 for them."

Lastly if they don't have `Mr` or `Mrs` in their name then we will get the avarage age of people who don't and use that for their missing `Age` value. 

In [11]:
def get_age_averages(df):
    total_mrs = 0
    total_mr = 0
    sum_mrs = 0
    sum_mr = 0
    for index, row in df.iterrows():
        if 'mrs' in row['Name'].lower() and not np.isnan(row['Age']):
            sum_mrs += row['Age']
            total_mrs += 1
        elif 'mr' in row['Name'].lower() and not np.isnan(row['Age']):
            sum_mr += float(row['Age'])
            total_mr += 1
    return round(sum_mrs/total_mrs, 2), round(sum_mr/total_mrs, 2)

In [12]:
def fill_age(df, mean_age_mrs, mean_age_mr):
    for index, row in df.iterrows():
        if 'mrs' in row['Name'].lower() and np.isnan(row['Age']):
            df.loc[index, 'Age'] = mean_age_mrs
        elif 'mr' in row['Name'].lower() and np.isnan(row['Age']):
            df.loc[index, 'Age'] = mean_age_mr
        elif not 'mrs' in row['Name'].lower() and not 'mr' in row['Name'].lower() and np.isnan(row['Age']) and row['Parch'] ==0:
            df.loc[index, 'Age'] = 0
    return df

In [13]:
mean_age_mrs, mean_age_mr = get_age_averages(train)

In [14]:
mean_age_mrs

35.41

In [15]:
train_age_filled = fill_age(train.copy(deep=True), mean_age_mrs, mean_age_mr)

In [16]:
train_age_filled

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C


In [33]:
train_age_filled.info() # info on data with filling in missing values

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Name      889 non-null    object 
 3   Sex       889 non-null    object 
 4   Age       876 non-null    float64
 5   SibSp     889 non-null    int64  
 6   Parch     889 non-null    int64  
 7   Fare      889 non-null    float64
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 101.7+ KB



<!-- The tables above tell us a couple of things. For starters we have see that we have 12 columns which are PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked. Lets breakdown each of them and see what might be useful for training our model. 

PassengderId: This column might not be vary useful as it is simply a id assigned by the dataset. We will drop this column before training. 

Survived: This column is our labels and is important since this is a supervised machine learning problem. If we wanted to go with a clustering/unsupervised learning we could drop this. For the purpose of the notebook we will be keeping this. 

Pclass: On kaggle it says that this columns serves "A proxy for socio-economic status". Where 1 is upper, 2 is middle and 3 is upper. This will be important for our model

Name: This is simply the name of the passenger. This could be important as some family last names could mean that they are from a wealthier family and therfore might have a higher chance of surviving. For this notebook we will most likely drop it. 

Sex/Age: The sex of the passenger. Since women and children were  prioritized in the case of an emergency this would helpful to determined who would survive. 

SibSp: In the kaggle description of the dataset it says that sibsp is "# of siblings / spouses aboard the Titanic". This could be helpful. 

Parch: In the kaggle description of the dataset it says that parch is "# of parents / children aboard the Titanic". This will also be helpful. 

Ticket: Simply the ticket ID so we can remove this.

Fare: How much they paid for their ticket. This can be useful as maybe workers did not pay for ticket and upper class people payed for their tickets. We will keep this column. 

Cabin: This is the cabin they were staying at. This could be useful but there are several missing values in this column so for now we will ignore it. 

Embarked: The Location of where they embarked. This could be important to determine who survived. For examples people who embarked at a certain location might be workers and others might be upper class familiies. This could be useful.  -->

In [34]:
train.info() # info on data without filling in missing values

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Name      889 non-null    object 
 3   Sex       889 non-null    object 
 4   Age       712 non-null    float64
 5   SibSp     889 non-null    int64  
 6   Parch     889 non-null    int64  
 7   Fare      889 non-null    float64
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 101.7+ KB


In [19]:
# # get info on the remaining dataframe
# train.info()

Our dataframe now only contains the columns that we want. However from the table above we can see that we still have quite a few missing values in a couple of columns. For example `Age` has quite a few missing values. `Embarked` has some missing values but nothing major. We now have to figure out what to do with the missing values for age. Lets first check how many missing values we have exactly. 

In [20]:
# count missing values in columns
# train.isnull().sum()

We have a total of 177 missing values. We have a couple of options on what to do for the missing values. We can either drop the rows of the missing values. In the case of embarked we probably will becuase it is two rows. For the case of age however we might want to try to fill the missing values because we would loose a lot of data if we dropped those rows. 

I will be doing both approaches. One where we drop the rows that have missing values and one where we fill in the missing values. At the end we will see if there is really a difference and if it was worth filling in the missing values. First lets go look into our missing values to decide what approach we will take. 

In [21]:
# get the rows that have missing values and create dataframe
# missing_vals_df = train[train.isna().any(axis=1)]

In [22]:
# show the first 15 samples in the dataframe of missing values
# missing_vals_df.head(15) 

From the table above we can see that from the first 15 samples most of the Pclass are 3. Lets count how many of the missing values are of Pclass 3. 

In [23]:
# in missing values count the pclass frequencies
# missing_vals_df['Pclass'].value_counts()

In [24]:
# missing_vals_df['Survived'].value_counts()

In [25]:
# missing_vals_df['Parch'].value_counts()

In [26]:
# from the dataframe containg missing age values get a frequency count on the fare
# missing_vals_df['Fare'].value_counts()

A lot of the missing values are from the Pclass 3. We are going to see if we can get more info as to why the age on some of these is unkown. Once we can make a guess we will then see if we can fill the values in. 

In [27]:
# def get_pclass(df, n):
#     """
#     seperate non age null dataframes into dataframes by their pclass (1,2,3)
#     """
#     dataframes = []
#     for i in range(1,n+1):
#         not_null = df[df['Age'].notnull()]
#         not_null_filtered = not_null[not_null['Pclass'] == i]
#         dataframes.append(not_null_filtered)
#     return dataframes

In [28]:
# pclass_df = get_pclass(train, 3)

In [29]:
# pclass_df.append(train[train['Age'].notnull()])

In [30]:
# pclass_df[1]['Age']

In [31]:
# plt.scatter(pclass_df[3]['Fare'], pclass_df[3]['Pclass'])
# plt.show()